# LanceDB

>[LanceDB](https://lancedb.com/) is an open-source database for vector-search built with persistent storage, which greatly simplifies retrevial, filtering and management of embeddings. Fully open source.

This notebook shows how to use functionality related to the `LanceDB` vector database based on the Lance data format.

In [ ]:
! pip install -U langchain-openai langchain-community

In [ ]:
! pip install lancedb

We want to use OpenAIEmbeddings so we have to get the OpenAI API Key. 

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
! rm -rf /tmp/lancedb

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import LanceDB
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()

documents = CharacterTextSplitter().split_documents(documents)
embeddings = OpenAIEmbeddings()

##### For LanceDB cloud, you can invoke the vector store as follows :


```python
db_url = "db://lang_test" # url of db you created
api_key = "xxxxx" # your API key
region="us-east-1-dev"  # your selected region

vector_store = LanceDB(
    uri=db_url,
    api_key=api_key,
    region=region,
    embedding=embeddings,
    table_name='langchain_test'
    )
```


In [4]:
docsearch = LanceDB.from_documents(documents, embeddings)
query = "What did the president say about Ketanji Brown Jackson"


In [5]:
docs = docsearch.similarity_search_with_relevance_scores(query)
print("relevance score - ",docs[0][1])
print("text- ",docs[0][0].page_content)

relevance score -  0.7066359758561034
text-  They were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. 

Officer Mora was 27 years old. 

Officer Rivera was 22. 

Both Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. 

I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. 

I’ve worked on these issues a long time. 

I know what works: Investing in crime prevention and community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety. 

So let’s not abandon our streets. Or choose between safety and equal justice. 

Let’s come together to protect our communities, restore trust, and hold law enforcement accountable. 

That’s why the Justice Department required body cameras, banned chokeholds, and restricted no-knock warrants for it

In [6]:
docs = docsearch.similarity_search_with_score('txt')
print("text- ",docs[0][0].page_content)

text-  My administration is providing assistance with job training and housing, and now helping lower-income veterans get VA care debt-free.  

Our troops in Iraq and Afghanistan faced many dangers. 

One was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. 

When they came home, many of the world’s fittest and best trained warriors were never the same. 

Headaches. Numbness. Dizziness. 

A cancer that would put them in a flag-draped coffin. 

I know. 

One of those soldiers was my son Major Beau Biden. 

We don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. 

But I’m committed to finding out everything we can. 

Committed to military families like Danielle Robinson from Ohio. 

The widow of Sergeant First Class Heath Robinson.  

He was born a soldier. Army National Guard. Combat medic in Kosovo and Iraq. 

Stationed near Baghdad, 

Additionaly, to explore the table you can load it into a df or save it in a csv file: 
```python
tbl = docsearch.get_table()
print("tbl:", tbl)
pd_df = tbl.to_pandas()
# pd_df.to_csv("docsearch.csv", index=False)

# you can also create a new vector store object using an older connection object:
vector_store = LanceDB(connection=tbl, embedding=embeddings)
```

In [7]:
docs = docsearch.similarity_search('txt')

print(docs[0].page_content)

My administration is providing assistance with job training and housing, and now helping lower-income veterans get VA care debt-free.  

Our troops in Iraq and Afghanistan faced many dangers. 

One was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. 

When they came home, many of the world’s fittest and best trained warriors were never the same. 

Headaches. Numbness. Dizziness. 

A cancer that would put them in a flag-draped coffin. 

I know. 

One of those soldiers was my son Major Beau Biden. 

We don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. 

But I’m committed to finding out everything we can. 

Committed to military families like Danielle Robinson from Ohio. 

The widow of Sergeant First Class Heath Robinson.  

He was born a soldier. Army National Guard. Combat medic in Kosovo and Iraq. 

Stationed near Baghdad, just ya

In [8]:
print(docs[0].page_content)

My administration is providing assistance with job training and housing, and now helping lower-income veterans get VA care debt-free.  

Our troops in Iraq and Afghanistan faced many dangers. 

One was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. 

When they came home, many of the world’s fittest and best trained warriors were never the same. 

Headaches. Numbness. Dizziness. 

A cancer that would put them in a flag-draped coffin. 

I know. 

One of those soldiers was my son Major Beau Biden. 

We don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. 

But I’m committed to finding out everything we can. 

Committed to military families like Danielle Robinson from Ohio. 

The widow of Sergeant First Class Heath Robinson.  

He was born a soldier. Army National Guard. Combat medic in Kosovo and Iraq. 

Stationed near Baghdad, just ya

In [9]:
print(docs[0].metadata)

{'source': '../../how_to/state_of_the_union.txt'}


Adding images 

In [ ]:
! pip install -U langchain-experimental

In [ ]:
! pip install open_clip_torch torch

In [11]:
! rm -rf '/tmp/multimmodal_lance'

In [12]:
from langchain_experimental.open_clip import OpenCLIPEmbeddings

In [13]:
import requests
import os

# List of image URLs to download
image_urls = [
    "https://github.com/raghavdixit99/assets/assets/34462078/abf47cc4-d979-4aaa-83be-53a2115bf318",
    "https://github.com/raghavdixit99/assets/assets/34462078/93be928e-522b-4e37-889d-d4efd54b2112",
]

texts = ['bird',
         'dragon']

# Directory to save images
dir_name = './photos/'

# Create directory if it doesn't exist
os.makedirs(dir_name, exist_ok=True)

image_uris = []
# Download and save each image
for i, url in enumerate(image_urls, start=1):
    response = requests.get(url)
    path = os.path.join(dir_name, f'image{i}.jpg')
    image_uris.append(path)
    with open(path, 'wb') as f:
        f.write(response.content)

In [14]:
from langchain_community.vectorstores import LanceDB
vec_store = LanceDB(uri='/tmp/multimmodal_lance',embedding=OpenCLIPEmbeddings())

In [15]:
vec_store.add_images(uris=image_uris)

['56a12f21-a978-471a-88cc-df325c9dd5b8',
 'fd629993-409e-4f68-89c2-7ec34212aed3']

In [16]:
vec_store.add_texts(texts)

['56681e66-ec48-461a-9519-e2686839a058',
 '89d79434-1b1d-4833-9d18-0e05e0ae96b1']

In [17]:
img_embed = vec_store._embedding.embed_query('bird')

In [18]:
vec_store.similarity_search_by_vector(img_embed)[0]

Document(page_content='bird', metadata={'id': '56681e66-ec48-461a-9519-e2686839a058'})